# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import datetime
import api_token  # handle iexcloud API tokens

env = "sandbox"  # "cloud"
BASE_URL, IEX_CLOUD_API_TOKEN = api_token.get_base_url(env=env), api_token.get_api_token(env=env)

In [2]:
TOP_N = 50

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
# stocks = pd.read_csv('sp_500_stocks.csv')

df = pd.read_csv('nasdaq_100_stocks.csv.gz', 
                 compression="gzip", 
                 names=["Ticker", "Company"], 
                 skiprows=[0])
stocks = df["Ticker"].values.tolist()

In [4]:
",".join(stocks)

'ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,ADI,ANSS,AAPL,AMAT,ASML,ADSK,ADP,BIDU,BIIB,BMRN,BKNG,AVGO,CDNS,CDW,CERN,CHTR,CHKP,CTAS,CSCO,CTXS,CTSH,CMCSA,CPRT,COST,CSX,DXCM,DOCU,DLTR,EBAY,EA,EXC,EXPE,FB,FAST,FISV,FOXA,FOX,GILD,IDXX,ILMN,INCY,INTC,INTU,ISRG,JD,KDP,KLAC,KHC,LRCX,LBTYA,LBTYK,LULU,MAR,MXIM,MELI,MCHP,MU,MSFT,MRNA,MDLZ,MNST,NTES,NFLX,NVDA,NXPI,ORLY,PCAR,PAYX,PYPL,PEP,PDD,QCOM,REGN,ROST,SGEN,SIRI,SWKS,SPLK,SBUX,SNPS,TMUS,TTWO,TSLA,TXN,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,XEL,XLNX,ZM'

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [5]:
symbol = 'AAPL'
api_url = f'{BASE_URL}/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2164318918555,
 'week52high': 135.05,
 'week52low': 57.31,
 'week52change': 0.797312775358145,
 'sharesOutstanding': 17576306762,
 'float': 0,
 'avg10Volume': 89011660,
 'avg30Volume': 93941414,
 'day200MovingAvg': 109.24,
 'day50MovingAvg': 123.25,
 'employees': 137400,
 'ttmEPS': 3.42,
 'ttmDividendRate': 0.820445828029047,
 'dividendYield': 0.006734258477888977,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-29',
 'nextEarningsDate': '0',
 'peRatio': 37.12398469359822,
 'beta': 0,
 'maxChangePercent': 47.76635058021345,
 'year5ChangePercent': 3.706024152891394,
 'year2ChangePercent': 2.06946580462182,
 'year1ChangePercent': 0.8365291463817899,
 'ytdChangePercent': 0.710412717047324,
 'month6ChangePercent': 0.4793595236800443,
 'month3ChangePercent': 0.09944182917541013,
 'month1ChangePercent': 0.025378045898336758,
 'day30ChangePercent': 0.025552764729303235,
 'day5ChangePercent': 0.00135191986080778}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [6]:
data['year1ChangePercent']

0.8365291463817899

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks, 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'{BASE_URL}/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,ATVI,86.63,0.499429,N/A
1,ADBE,477.79,0.583094,N/A
2,AMD,94.22,1.343276,N/A
3,ALXN,122.13,0.107960,N/A
4,ALGN,508.57,0.848026,N/A
...,...,...,...,...
98,WBA,42.02,-0.259149,N/A
99,WDAY,225.74,0.332382,N/A
100,XEL,65.00,0.075202,N/A
101,XLNX,150.66,0.567125,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [9]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:(TOP_N+1)]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,TSLA,625.310,7.707228,N/A
1,MRNA,157.940,7.685031,N/A
2,ZM,407.850,5.495757,N/A
3,PDD,148.470,3.040724,N/A
4,DOCU,231.690,2.209554,N/A
5,MELI,1627.090,1.792517,N/A
6,JD,82.700,1.536473,N/A
7,NVDA,545.310,1.440877,N/A
8,AMD,94.220,1.343276,N/A
9,PYPL,222.000,1.037107,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [11]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\19197\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,TSLA,625.310,7.707228,31
1,MRNA,157.940,7.685031,124
2,ZM,407.850,5.495757,48
3,PDD,148.470,3.040724,132
4,DOCU,231.690,2.209554,84
5,MELI,1627.090,1.792517,12
6,JD,82.700,1.536473,237
7,NVDA,545.310,1.440877,35
8,AMD,94.220,1.343276,208
9,PYPL,222.000,1.037107,88


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [12]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'{BASE_URL}/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [13]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# # Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ATVI,84.70,N/A,0.507479,0.718447,0.174512,0.368932,0.073767,0.339806,0.096858,0.747573,N/A
1,ADBE,492.90,N/A,0.572914,0.757282,0.231504,0.475728,0.009751,0.165049,0.017612,0.378641,N/A
2,AMD,92.33,N/A,1.382424,0.92233,0.757937,0.932039,0.206725,0.718447,0.129921,0.834951,N/A
3,ALXN,121.33,N/A,0.108082,0.320388,0.106296,0.252427,0.153720,0.553398,-0.054170,0.0582524,N/A
4,ALGN,528.33,N/A,0.882579,0.902913,1.135278,0.980583,0.613130,0.951456,0.101853,0.76699,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
98,WBA,42.85,N/A,-0.263406,0.00970874,0.043333,0.15534,0.213356,0.728155,0.014671,0.320388,N/A
99,WDAY,229.06,N/A,0.331742,0.504854,0.268621,0.533981,0.063351,0.291262,0.002917,0.281553,N/A
100,XEL,66.00,N/A,0.074574,0.252427,0.012135,0.106796,-0.051343,0.106796,-0.131114,0.0291262,N/A
101,XLNX,149.46,N/A,0.563669,0.737864,0.624481,0.902913,0.483694,0.912621,0.135152,0.873786,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [14]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [18]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace=True)
hqm_dataframe = hqm_dataframe[:(TOP_N+1)]

<ipython-input-18-d8b1b5c16398>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace=True)


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [19]:
portfolio_input()

Enter the value of your portfolio:1000000


In [20]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
4,ALGN,528.330,37,0.882579,0.902913,1.135278,0.980583,0.613130,0.951456,0.101853,0.76699,0.900485
12,AMAT,89.100,220,0.530363,0.728155,0.627572,0.912621,0.636154,0.961165,0.250754,0.961165,0.890777
2,AMD,92.330,212,1.382424,0.92233,0.757937,0.932039,0.206725,0.718447,0.129921,0.834951,0.851942
16,BIDU,161.800,121,0.386363,0.601942,0.462898,0.805825,0.346305,0.893204,0.129892,0.825243,0.781553
35,DOCU,228.160,85,2.113434,0.961165,0.528268,0.864078,0.141314,0.504854,0.084316,0.718447,0.762136
13,ASML,467.040,41,0.607157,0.786408,0.371155,0.699029,0.265907,0.815534,0.086109,0.737864,0.759709
14,ADSK,284.880,68,0.576426,0.76699,0.280942,0.553398,0.233957,0.747573,0.135066,0.864078,0.73301
47,IDXX,456.780,42,0.843393,0.893204,0.545448,0.873786,0.259296,0.805825,0.002600,0.271845,0.711165
11,AAPL,126.700,154,0.830553,0.873786,0.479497,0.834951,0.097348,0.427184,0.025261,0.427184,0.640777
40,EXPE,123.090,159,0.106975,0.31068,0.584747,0.893204,0.304340,0.873786,0.028043,0.456311,0.633495


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [21]:
today = datetime.datetime.now()
xlsx_file = f"002-momentum_strategy_{today.strftime('%Y-%m-%d')}.xlsx"

In [22]:
xlsx_file

'002-momentum_strategy_2020-12-12.xlsx'

In [23]:
writer = pd.ExcelWriter(xlsx_file, engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 10, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [26]:
writer.save()